<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Liberaries

In [10]:
import numpy as np
import pickle
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler

## Reading the prepared dataset and split it

In [2]:
path_local='../Data/data68.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data68.csv'
data =pd.read_csv(path_local)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y63,y64,y65,y66,y67,y68,yaw,pitch,roll,img
0,279.05283,281.67548,285.68677,287.899540,284.69318,275.74564,261.69530,252.67186,252.65305,264.91882,...,309.33150,311.98444,318.51562,319.20294,317.46520,313.99393,69.062935,-48.795550,-31.882296,image03258
1,106.78756,107.11935,111.29989,115.847370,124.11620,140.81512,161.30240,185.51682,216.90814,245.06805,...,304.52728,305.67432,307.28482,319.07630,319.09350,316.20816,-13.392587,-6.686405,5.489456,image03247
2,123.74660,123.77142,123.46777,127.416306,139.26894,162.47030,190.68597,213.83716,222.32365,212.37135,...,316.44590,320.29510,330.61310,324.62158,320.40310,315.17682,-82.824690,-31.047533,61.351288,image03252
3,241.90388,241.49951,242.88312,242.789640,240.46445,236.20209,231.27866,229.64096,238.73349,256.18690,...,298.07925,300.00455,305.23682,304.31770,302.94162,300.31104,47.252384,-21.523932,0.100378,image03237
4,106.61653,113.84815,124.36699,134.661160,146.94208,164.04477,182.50476,206.71774,243.18040,279.36014,...,316.60840,315.73355,318.12878,319.93948,322.06128,323.02048,3.578812,5.349906,-7.209053,image03250


In [4]:
X = data.iloc[:,:-4]
X.head(1)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y59,y60,y61,y62,y63,y64,y65,y66,y67,y68
0,279.05283,281.67548,285.68677,287.89954,284.69318,275.74564,261.6953,252.67186,252.65305,264.91882,...,323.64362,315.69916,305.01852,307.28442,309.3315,311.98444,318.51562,319.20294,317.4652,313.99393


In [5]:
y = data.iloc[:,-4:-1]
y.head(1)

,yaw,pitch,roll
0,69.062935,-48.79555,-31.882296


## Scaling

In [6]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.20, random_state=42, shuffle=True)

## **Trial #1:**

- MultiOutputRegressor with RandomForestRegressor


> https://www.r-bloggers.com/2021/10/multi-output-regression-using-sklearn/





In [8]:
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=12, random_state=42,n_estimators=300,min_samples_split=4))
clf.fit(x_train, y_train)
clf.score(x_train, y_train, sample_weight=None)

0.9182871059320766

In [9]:
y_pred= clf.predict(x_test)
clf.score(x_test, y_test, sample_weight=None)

0.8493021742209249

In [11]:
# save model
with open('../SavedModels/model_Rm.pkl', 'wb') as file:
        pickle.dump(clf, file)

## **Trial #2:**
  - SVM

  - https://machinelearningmastery.com/multi-output-regression-models-with-python/

In [12]:
# define base model
model = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1)
# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
wrapper.fit(x_train, y_train)
wrapper.predict(x_test)

array([[ -1.47333357, -13.51525501,   0.4511346 ],
       [ -0.6954016 , -12.21263743,   3.54889648],
       [ 76.39309442,  18.67392432,  20.40252423],
       ...,
       [ -2.25644212, -17.34918929,  -3.21295915],
       [ 16.80148234,   2.87005993,  20.35237089],
       [-54.18071901, -11.10747594,  20.74042037]])

In [14]:
print(wrapper.score(x_train, y_train, sample_weight=None))
print(wrapper.score(x_test, y_test, sample_weight=None))

0.9431755167202324
0.8297890379076357


In [15]:
# save model
with open('../SavedModels/model_svm.pkl', 'wb') as file:
        pickle.dump(wrapper, file)

## **Trial #3:**

- GradientBoostingRegressor

In [16]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42,n_estimators=100,learning_rate=0.05)
model = MultiOutputRegressor(estimator=gbr)

In [17]:
model.fit(x_train, y_train)
score = model.score(x_train, y_train)
print("Training score:", score)
ypred = model.predict(x_test)
model.score(x_test, y_test, sample_weight=None)

Training score: 0.98825433442155


0.8576349750525792

In [18]:
# save model
with open('../SavedModels/model_GB.pkl', 'wb') as file:
        pickle.dump(model, file)